Final goal -> being able to choose setpoints which result in lower energy consumption but without the dropping production

We need to devide up the task, starting with one controller we allow the model to make one of three moves defined as 

### ACTIONS: 
(step up, step down, stay) 

by a predefined increment. (We can add increment size later. )

The action should be a responce of the current state of the model (model is here the k-spice simulator) which is described through a set of 

### Observations: 
Input - This is obvious, the parameter we intend to change.

Our ultimate goal is to learn a desirable policy between (state, action) pairs and to achieve this we need to reward good actions and punish bad actions. Hence, we construct a reward consisting of 

### Reward(Energy consumption)

and we define a set of breakpoints which implies the end of an episode, i.e. after an action if this occurs - reset and try again. 

### Breakpoints: 
Alarmlimits reached 
Time out - did not reach all the setpoints in a predefined timelimit



In [23]:
#Kspice
import sys
sys.path.append(r"C:\Program Files (x86)\Kongsberg\K-Spice\bin64") #add to path to allow kspice import
import kspice # if import error, check correct python version (3.11)

#Basic functionality
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#ML
import torch 

from enviroment import Sim
from DQN import DQN

In [24]:
project_path = r"C:\Appl\K-Spice-Projects\Kristin_v23" #Specify path to downloaded project.
_ = kspice.Simulator(project_path) #Create instance of project
timeline = _.activate_timeline("Engineering") # Select the avaliable engineering timeline
app = "Topside" # We only make changes to the topside module NOTE: From software we can #deactivate Wells and Power in ESS model, can this be done from python? (If it increases speed)
timeline.initialize() #

In [25]:
timeline.load_model("KristinMaria_master") #Load model
timeline.load_parameters("KristinMaria_master") # load fixed parameters
timeline.load_initial_condition("KristinMaria_master") # Load initial conditions

In [26]:
env = Sim(timeline, app)
env.import_variables("xl_tester2.xlsx")

In [27]:
env.check_df("Global KPI")

In [28]:
env.timeline.set_speed(10)
env.timeline.run()

In [43]:
import random 
from collections import deque 


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [29]:
def select_action(state):
    """ To ensure the model explores new spaces we will sometimes choose actions randomly. If not random we choose the action which result in the highest expected reward. 
    Choosing a random action will decay exponientially throughout learning. 
    """
    EPS_START = 0.9
    EPS_END = 0.05
    EPS_DECAY = 1000

    global steps_done
    sample = np.random.rand(1)
    eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor(env.sample("action"), dtype=torch.float32) # random action

In [30]:
timeline.achieved_speed

1.0819176892697302

In [10]:
state = env.reset(pause = True)

In [32]:
n_states = sum([len(state[i]) for i in range(len(state))]) # the flattened state output will be the input layer to the network
n_actions = 3 # (up, down, stay)

In [33]:
import torch.nn as nn
import torch.nn.functional as F


class DQN(nn.Module):
    def __init__(self, n_states, n_actions):
        super(DQN, self).__init__()

        # Let first try with a "simple" MLP with 1 hidden layer
        self.layer1 = nn.Linear(n_states, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)


    def forward(self, x):
        # used in __call__
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [34]:
from collections import namedtuple, deque


Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


BATCH_SIZE = 128 # number of transitions sampled from the replay buffer
GAMMA = 0.99 # The factor in the discounted cumulative reward, we care about the rewards in the future but with a discount factor gamma. 
TAU = 0.005 # The target network should change slowly to improve stability - hence a soft update, with the TAU factor, of the target weights is used.  
LR = 1e-4 #Learing rate of optimizer

policy_net = DQN(n_states, n_actions)
target_net = DQN(n_states, n_actions)

target_net.load_state_dict(policy_net.state_dict())

optimizer = torch.optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(100)


steps_done = 0

In [35]:
import itertools
state_flat = np.array(list(itertools.chain.from_iterable(state)))
state_tensor = torch.tensor(state_flat, dtype=torch.float32).unsqueeze(0) # make torch tensor and add axis

In [36]:
steps_done = 0
action = select_action(state_tensor)
print(action)

tensor([0.])


In [37]:
#NOTE from this point I have not generalized to multiple controllers but only consider one. 

env.step(action)

S23TT1004 has reached its setpoint 29.98071560364008 [C]
[]


((array([29.9807156]),
  array([ 7.40843203e+01, -1.87935296e+06, -8.20399993e+05, -1.12496959e+06]),
  array([30.])),
 -1,
 None,
 False)

In [52]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        print(len(memory), "len memory")
        return
    print(len(memory), 'len after')
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net

    print(policy_net(state_batch), action_batch)
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [39]:
steps_done = 0
#select_action(state)

policy_net(state_tensor).max(1).indices.view(1,1) #return index of largest column as (1,1) tensor

tensor([[1]])

In [40]:
state = env.reset()
state_flat = np.array(list(itertools.chain.from_iterable(state)))
state = torch.tensor(state_flat, dtype=torch.float32).unsqueeze(0) # make torch tensor and add axis

action = select_action(state)
observation, reward, terminated, truncated = env.step(action)
reward = torch.tensor([reward])
done = terminated or truncated

S23TT1004 has reached its setpoint 29.09824588214684 [C]
[]


In [53]:
BATCH_SIZE = 2 # let see if the optimizer works

if terminated:
    next_state = None
else:
    _ = np.array(list(itertools.chain.from_iterable(observation)))
    next_state = torch.tensor(_, dtype=torch.float32).unsqueeze(0) # make torch tensor and add axis

# Store the transition in memory
memory.push(state, action, next_state, reward)

# Move to the next state
state = next_state

# Perform one step of the optimization (on the policy network)
optimize_model()

8 len after
tensor([[-34635.7812, -25968.5195, -38436.2812],
        [-34635.7969, -25968.5117, -38436.2773]], grad_fn=<AddmmBackward0>) tensor([-1., -1.])


RuntimeError: gather(): Expected dtype int64 for index

In [58]:
torch.tensor(np.array([[1,1,1]])).gather(1, torch.tensor(np.array([1,1])))

RuntimeError: gather(): Expected dtype int64 for index